In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from itertools import chain
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
import numpy as np
import re
from numpy import random

In [2]:
all_countries=pd.read_csv("C:/Users/homa/Python-Notbook/TradeMap/all_countries.csv")
Product_Code=pd.read_csv("C:/Users/homa/Python-Notbook/TradeMap/Product_Codem.csv")
all_countries.head(2)

,Country,Unnamed: 1,Country1,Unnamed: 3
0,'Afghanistan,NaN,Afghanistan,NaN
1,'Albania,NaN,Albania,NaN


In [22]:
driver= webdriver.Chrome()
driver.get("https://www.trademap.org/Index.aspx")
#driver.maximize_window()
waits= WebDriverWait(driver, 30)

In [4]:
def login():
    try:
        logintext=driver.find_element(By.CSS_SELECTOR ,("#ctl00_MenuControl_Label_Login"))
        if logintext.text=='Login': 
                
            login=waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_MenuControl_Label_Login')))
            login.click()
            url= driver.current_url
            if url!='https://www.trademap.org/Index.aspx': 
                getemail= waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#Username')))
                getemail.send_keys('homa.abedinzadeh@gmail.com')
                #time.sleep(random.randint(5,20))
                getpassword= waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#Password')))
                getpassword.send_keys('Aa@97019701')
                #time.sleep(random.randint(5,20))
                waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR,'body > div.container > div > div:nth-child(2) > div > div > div > form > fieldset > div:nth-child(4) > div > button'))).click()
            
    
    except Exception as e:
                print(f"Error selecting option : {e}")


In [5]:
def delete():
    waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_Image_deleteProduct'))).click()
    waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_Image_DeleteCountry'))).click()  

In [23]:
df = pd.DataFrame()
temp = pd.DataFrame()
for i in range(len(Product_Code)):
    login()
    delete()
    inputproduct = waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_RadComboBox_Product_Input')))
    inputproduct.send_keys(Product_Code['ProductCode'].iloc[i])
    time.sleep(random.randint(3,6))
    proclick = waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_RadComboBox_Product_c0'))).click()
    time.sleep(random.randint(2,3))
    btn = waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_Button_TradeIndicators'))).click()

    dropdown_element = waits.until(EC.visibility_of_element_located((By.ID, 'ctl00_NavigationControl_DropDownList_Country')))  # Replace with actual ID     
    select = Select(dropdown_element)
    options = select.options
    for j in range(len(options)):  
            print(j)
            try:
                
                dropdown_element = waits.until(EC.visibility_of_element_located((By.ID, 'ctl00_NavigationControl_DropDownList_Country')))
                select = Select(dropdown_element)
                options = select.options
                option_text = options[j].text

                if option_text != 'All' or option_text.find('elsewhere ')!=-1  :
                    select.select_by_visible_text(option_text)
                    # تغییر تعداد رکوردها به 300
                    incshow = waits.until(EC.element_to_be_clickable((By.XPATH, "//select[@id='ctl00_PageContent_GridViewPanelControl_DropDownList_PageSize']")))
                    drop = Select(incshow)
                    drop.select_by_value('300')





                # استخراج جدول
                    all_tables = pd.read_html(driver.page_source, attrs={'id': 'ctl00_PageContent_MyGridView1'})
                    temp = all_tables[0]

                    if temp.shape[0] != 1:
                        temp = temp.iloc[1:, 1:].iloc[:, :-4]
                        new_header = temp.iloc[0]
                        temp = temp[1:]
                        temp.columns = new_header

                        # افزودن اطلاعات کشور و کد محصول
                        temp['Importer'] = option_text
                        temp['productcode'] = re.sub(r'[^\w]', ' ', Product_Code['ProductCode'].iloc[i])
                        temp.drop(temp[temp['Exporters'] == 'Total'].index, inplace=True)

                        # تغییر نام ستون‌ها
                        average = temp.columns[15]
                        sharein = temp.columns[3]
                        temp.rename(columns={average: 'Average tariff%', sharein: 'ShareinCounryImports'}, inplace=True)

                        # افزودن به DataFrame اصلی
                        df = pd.concat([df, temp], axis=0)
                        temp = pd.DataFrame()

                        print(f"Selected option: {option_text}, {j}")
                    time.sleep(random.randint(1,3))
                if (j+1)%10 ==0:
                        #driver.quit()
                        #driver= webdriver.Chrome()
                        waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_MenuControl_Label_Menu_Home'))).click()
                        time.sleep(random.randint(2,5))
                        login()
                        delete()
                        time.sleep(random.randint(3,5))
                        inputproduct = waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_RadComboBox_Product_Input')))
                        inputproduct.send_keys(Product_Code['ProductCode'].iloc[i])
                        time.sleep(random.randint(3,6))
                        proclick = waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_RadComboBox_Product_c0'))).click()
                        time.sleep(random.randint(2,3))
                        btn = waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_Button_TradeIndicators'))).click()
                        
            
            except Exception as e:
                print(f"Error selecting option {j}: {e}")
                site = driver.current_url
                if site!= 'https://www.trademap.org/Index.aspx':
                    driver.get('https://www.trademap.org/Index.aspx')
                login()
                delete()
                time.sleep(random.randint(3,5))
                inputproduct = waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_RadComboBox_Product_Input')))
                inputproduct.send_keys(Product_Code['ProductCode'].iloc[i])
                time.sleep(random.randint(3,6))
                proclick = waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_RadComboBox_Product_c0'))).click()
                time.sleep(random.randint(2,3))
                btn = waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_Button_TradeIndicators'))).click()
                j-=1

waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_MenuControl_Label_Menu_Home'))).click()
time.sleep(30)
driver.refresh()
i+=1


Error selecting option : Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=129.0.6668.71)
Stacktrace:
	GetHandleVerifier [0x00007FF65142B645+29573]
	(No symbol) [0x00007FF6513A0470]
	(No symbol) [0x00007FF65125B6EA]
	(No symbol) [0x00007FF65122FCD5]
	(No symbol) [0x00007FF6512DEF67]
	(No symbol) [0x00007FF6512F7FC1]
	(No symbol) [0x00007FF6512D70A3]
	(No symbol) [0x00007FF6512A12DF]
	(No symbol) [0x00007FF6512A2441]
	GetHandleVerifier [0x00007FF65175C58D+3375821]
	GetHandleVerifier [0x00007FF6517A7987+3684039]
	GetHandleVerifier [0x00007FF65179CDAB+3640043]
	GetHandleVerifier [0x00007FF6514EB7C6+816390]
	(No symbol) [0x00007FF6513AB77F]
	(No symbol) [0x00007FF6513A75A4]
	(No symbol) [0x00007FF6513A7740]
	(No symbol) [0x00007FF65139659F]
	BaseThreadInitThunk [0x00007FFAE371257D+29]
	RtlUserThreadStart [0x00007FFAE43CAF08+40]



NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=129.0.6668.71)
Stacktrace:
	GetHandleVerifier [0x00007FF65142B645+29573]
	(No symbol) [0x00007FF6513A0470]
	(No symbol) [0x00007FF65125B6EA]
	(No symbol) [0x00007FF65122FCD5]
	(No symbol) [0x00007FF6512DEF67]
	(No symbol) [0x00007FF6512F7FC1]
	(No symbol) [0x00007FF6512D70A3]
	(No symbol) [0x00007FF6512A12DF]
	(No symbol) [0x00007FF6512A2441]
	GetHandleVerifier [0x00007FF65175C58D+3375821]
	GetHandleVerifier [0x00007FF6517A7987+3684039]
	GetHandleVerifier [0x00007FF65179CDAB+3640043]
	GetHandleVerifier [0x00007FF6514EB7C6+816390]
	(No symbol) [0x00007FF6513AB77F]
	(No symbol) [0x00007FF6513A75A4]
	(No symbol) [0x00007FF6513A7740]
	(No symbol) [0x00007FF65139659F]
	BaseThreadInitThunk [0x00007FFAE371257D+29]
	RtlUserThreadStart [0x00007FFAE43CAF08+40]
